<a href="https://colab.research.google.com/github/kirajcg/pyscbwrapper/blob/master/pyscbwrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inledning

Vi vill konstruera en graf över invånartätheten i varje län i Sverige över tid. Till vår hjälp har vi SCB:s data och Python, och vår målsättning är att inte lämna Pythonmiljön ens för att hämta datan. Lyckligtvis finns det ett särskilt utvecklat paket för detta ändamål, kallat pyscbwrapper. 

# Installation

pyscbwrapper är ett slags gränssnitt till Statistikdatabasen, skrivet i Python 3. För att köra detta gränssnitt behöver vi först installera och importera det. (Om du inte har tillgång till Python så kan du köra det via Colab genom att klicka på knappen ovan.)

In [0]:
!pip install -q pyscbwrapper
from pyscbwrapper import SCB

# Initiering

SCB är nu en klass vi har importerat, som innehåller funktioner för att navigera i Statistikdatabasen och hämta både metadata och data från den. För att använda SCB-klassen behöver vi först initiera ett objekt från den. Då behövs ett obligatoriskt argument: språk för datan och metadatan. Engelska och svenska stöds. Vi väljer svenska: 

In [0]:
scb = SCB('sv')

# Navigering och metadata

Nu kan vi titta på toppnoden i det träd som är Statistikdatabasens metadata: 

In [0]:
scb.info()

Vi kan nu gå ned i trädet med hjälp av id-taggen i metadatan vi fick ut. Säg att vi är intresserade av befolkningsstatistiken: 

In [0]:
scb.go_down('BE')

För att nu få fram metadata om befolkningsstatistiken kör vi återigen funktionen info():

In [0]:
scb.info()

Vi kan fortsätta gå ned i trädet: 

In [0]:
scb.go_down('BE0001')
scb.info()

Oops! Vi ville inte titta på namnstatistiken utan befolkningsstatistiken. Vi går upp ett steg igen och vidare ned till rätt nod: 

In [0]:
scb.go_up()
scb.go_down('BE0101')
scb.info()

# Direktväg till specifika noder

Om vi vet vart vi vill i trädet är det möjligt att initiera ett objekt med id-taggarna som extra variabler:

In [0]:
scb = SCB('sv', 'BE', 'BE0101')
scb.info()

Som du ser hamnar vi direkt i befolkningsstatistiken.

---


Objektets specifika initiering hindrar oss dock inte från att navigera i trädet: 

In [0]:
scb.go_up()
scb.info()

Hursomhelst, vi går tillbaka direkt till befolkningstäthet:

In [0]:
scb.go_down('BE0101', 'BE0101C')
scb.info()

Nu finns det bara en nod att gå till så vi gör det: 

In [0]:
scb.go_down('BefArealTathetKon')
scb.info()

Notera hur metadatan skiljer sig åt från föregående noder: Nyckelordet variables finns, och detta indikerar att vi står i en slutnod. Härifrån kan vi därför hämta den faktiska datan. 

---

Det är inte är nödvändigt att kalla info() efter varje go_down() men det är en bra idé att göra ändå, om man inte är väldigt säker på hur databasen ser ut. 

# Hämta data

Nu när vi är i en slutnod kan vi titta på vilka variabler som finns, och deras värdemängder: 

In [0]:
scb.get_variables()

När vi har dessa kan vi välja vad vi är intresserade av och skapa en jsonfråga. Säg att vi är intresserade av invånare per kvadratkilometer i Örebro län för de senaste fem åren. 

In [0]:
scb.set_query(region=["Örebro län"], 
              tabellinnehåll=["Invånare per kvadratkilometer"], 
              år=["2014", "2015", "2016", "2017", "2018"])

Nu kan vi kolla hur frågan ser ut: 

In [0]:
scb.get_query()

Frågan blir automatiskt formaterad på rätt sätt för att hämta datan från API:et. Vi hämtar datan: 

In [0]:
scb.get_data()

Detta är samma data som man kan hämta från Statistikdatabasen på hemsidan. Genom en funktion i pyscbwrapper kan vi få ut URL:en till sidan med datan: 

In [0]:
scb.get_url()

Datan kan förstås också hämtas utan pyscbwrapper, genom att navigera till URL:en ovan. Där markerar vi "Invånare per kvadratkilometer", väljer indelning Län och markerar Örebro län, samt markerar de fem senaste åren. På nästa sida kan vi då klicka på "API för denna tabell" och få fram frågan och en URL att posta frågan till med hjälp av exempelvis paketet requests, men vi måste ändra "format": "px" till "format": "json". 

In [0]:
import requests
import json

session = requests.Session()

query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionLän99EjAggr",
        "values": [
          "18"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "BE0101U1"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2014",
          "2015",
          "2016",
          "2017",
          "2018"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

url = "https://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101C/BefArealTathetKon"

response = session.post(url, json=query)
response_json = json.loads(response.content.decode('utf-8-sig'))

response_json

Som du ser är det exakt samma data vi får ut. 

# Mer avancerade anrop

Nu när vi har sett hur datan ser ut kan vi plocka fram mer av den för att göra intressanta grafer. Eftersom vi redan är på rätt plats i API-strukturen behöver vi bara skapa en ny fråga. Säg att vi vill ha data från alla år från alla län. Vi kan först ta ut en lista över alla regioner, filtrera ut länen med hjälp av reguljära uttryck, och därefter använda listan i jsonfrågan: 

In [0]:
import re

regioner = scb.get_variables()['region']
r = re.compile(r'.* län')
lan = list(filter(r.match, regioner))

scb.set_query(region=lan,
              tabellinnehåll=["Invånare per kvadratkilometer"])

In [0]:
scb.get_query()

Det där är precis det frågan vi behöver nu. Vi tar ut datan och placerar den i en variabel så vi kan använda den senare: 

In [0]:
scb_data = scb.get_data()

I god sed tittar vi på datan innan vi gör någonting mer: 

In [0]:
scb_data

Själva datan vi söker ligger nu här: 

In [0]:
scb_uttag = scb_data['data']

Än en gång kontrollerar vi att vi har fått fram rätt data: 

In [0]:
scb_uttag

Nu behöver vi förstå strukturen på datan. Vi har fått ut en lista av dictionaries där den första variabeln key innehåller domänen (i detta fall län och år), och variabeln values innehåller värdet på undersökningsvariabeln (i detta fall invånare per kvadratkilometer). För att omforma detta till tidsserier som kan användas för visualisering krävs en del syntaktiska trick som beskrivs nedan. 

# Bearbetning av data

Vad vi söker är en tidsserie per län. Vi behöver därför strukturera om datan vi har fått ut innan vi kan göra någonting. Detta är utanför funktionaliteten hos pyscbwrapper, men vi kan lösa det med smidighet. En bra struktur vore en dictionary med län som nyckel och en annan dictionary som värde, där den inre dictionarien har år som nyckel och variabelvärde som värde. För detta tar vi fram listan över län som vi skapade tidigare och kopplar dem mot länkoderna, som vi kan plocka från dess plats i get_query(), i vårt fall 0. Genom att jämföra dessa koder, som är kopplade mot länsnamnen, med koderna i datan kan vi koppla länsnamnen mot datan. 

På så vis skapar vi strukturen vi vill ha, och passar på att göra värdena till numeriska. 

In [0]:
koder = scb.get_query()['query'][0]['selection']['values']

landic = {}
for i in range(len(koder)):
  landic[koder[i]] = lan[i]


landata = {}

for kod in landic:
  landata[landic[kod]] = {}
  for i in range(len(scb_uttag)):
    if scb_uttag[i]['key'][0] == kod:
      landata[landic[kod]][scb_uttag[i]['key'][1]] = \
      float(scb_uttag[i]['values'][0])


Det här blev ganska hackigt, så låt oss se om vad vi fick ut har den struktur vi sökte: 

In [0]:
landata

Det ser ju rätt ut. Nu kan vi loopa över nycklarna och plotta värdena med nyckel på x-axeln och värde på y-axeln. 

# Visualisering av data

Vi behöver numpy, pandas och matplotlib för detta. Vi installerar och importerar. 

In [0]:
!pip install -q matplotlib
!pip install -q pandas
!pip install -q numpy

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Nu kan vi ta fram en trevlig graf.

In [0]:
df = pd.DataFrame(landata)
df = df.reset_index()
df = df.rename(columns={"index":"År"})
ax = df.plot(x=df.index, xticks=np.arange(len(df.index)), colormap='hsv')
ax.set_xticklabels(df["År"], rotation=45)
plt.title("Befolkningstäthet per län")
plt.xlabel("År")
plt.ylabel("Invånare per kvadratkilometer")
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

Nu har vi vad vi ville, och vi lämnade aldrig Pythonmiljön!